In [21]:
'''
  code by Minho Ryu @bzantium
'''
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

char_arr = list('abcdefghijklmnopqrstuvwxyz')
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
n_class = len(word_dict) # number of class(=number of vocab)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
n_step = 3
n_hidden = 128

def make_batch(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

# Model
class TextLSTM(object):
    def __init__(self, sess, n_step, n_hidden, n_class):
        self.sess = sess
        self.n_step = n_step
        self.n_hidden = n_hidden
        self.n_class = n_class
        self._build_model()
    
    def _build_model(self):
        self.X = tf.placeholder(tf.float32, [None, self.n_step, self.n_class]) # [batch_size, n_step, n_class]
        self.Y = tf.placeholder(tf.float32, [None, self.n_class])         # [batch_size, n_class]
        
        W = tf.Variable(tf.random_normal([self.n_hidden, self.n_class]))
        b = tf.Variable(tf.random_normal([self.n_class]))
        
        cell = tf.nn.rnn_cell.BasicLSTMCell(self.n_hidden)
        outputs, states = tf.nn.dynamic_rnn(cell, self.X, dtype=tf.float32)
        outputs = tf.transpose(outputs, [1, 0, 2]) # [n_step, batch_size, n_hidden]
        outputs = outputs[-1]  # [batch_size, n_hidden]
        self.logits = tf.matmul(states[-1], W) + b # model : [batch_size, n_class]
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(0.001).minimize(self.cost)
        self.prediction = tf.cast(tf.argmax(self.logits, 1), tf.int32)

        tf.global_variables_initializer().run()
    
    def train(self, inputs, labels):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: inputs, self.Y: labels})
     
    def predict(self, inputs):
        return self.sess.run([self.prediction], feed_dict={self.X: inputs})
        

# Training
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
with tf.Session(config=run_config) as sess:
    input_batch, target_batch = make_batch(seq_data)
    model = TextLSTM(sess, n_step, n_hidden, n_class)
    for epoch in range(1000):
        loss, _ = model.train(input_batch, target_batch)
        if (epoch + 1) % 100 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    predict =  model.predict(input_batch)[0]

inputs = [sen[:3] for sen in seq_data]
for i, input in enumerate(inputs):
    print(input, '->', number_dict[predict[i]])

Epoch: 0100 cost = 0.019288
Epoch: 0200 cost = 0.004340
Epoch: 0300 cost = 0.001885
Epoch: 0400 cost = 0.001052
Epoch: 0500 cost = 0.000671
Epoch: 0600 cost = 0.000466
Epoch: 0700 cost = 0.000342
Epoch: 0800 cost = 0.000262
Epoch: 0900 cost = 0.000206
Epoch: 1000 cost = 0.000167
mak -> e
nee -> d
coa -> l
wor -> d
lov -> e
hat -> e
liv -> e
hom -> e
has -> h
sta -> r
